In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from ml_metrics import rmse
import pickle
from sklearn.linear_model import LinearRegression

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
#Filter for quantative features
quant =[f for f in df_train.columns if df_train.dtypes[f]!=("object")]
#Remove Id and SalesPrice
quant.remove('SalePrice')
quant.remove('Id')

#Filter for Qualitative
qualt =[f for f in df_train.columns if df_train.dtypes[f]==("object")]

In [4]:
df_quant = df_train[quant]

In [5]:
df_for_heatmap = df_quant.copy()
df_for_heatmap['SalePrice'] = df_train['SalePrice']

In [6]:
df_2=df_for_heatmap.drop(columns=['1stFlrSF'])

In [7]:
def SelectHighlyCorr_withTarget(df,corr):
    #return df with only high correlated features
    #Getting all correlations
    cor = df.corr()
    #Correlations with target
    cor_target = abs(cor["SalePrice"])
    relevant_features = cor_target[cor_target>=corr]

    rl = df_for_heatmap[relevant_features.index]
    
    return rl

df_3=SelectHighlyCorr_withTarget(df_2,0.1)

In [8]:
#sns.heatmap(df_3.corr(), cmap ='RdYlGn', linewidths = 0.2, annot = True)
#fig=plt.gcf()
#fig.set_size_inches(25,25)
#plt.show()

In [9]:
## Final Train df
final_train = df_3.copy()
final_train.drop(columns=['SalePrice'],inplace=True)

In [10]:
# filling NA
def fillingNA(df):
    df2 = df.fillna(final_train.median())
    return df2

final_train = fillingNA(final_train)

In [11]:
#Splitting data
#Final train is the final dataframe with removal of uneeded features
cols_to_select = final_train.columns
y = df_train['SalePrice']
x_train, x_validation, y_train, y_validation = train_test_split(final_train , y ,  train_size=0.8, test_size=0.2, random_state =20)

#Model
model = LinearRegression()
model.fit(x_train, y_train)

#Prediction
y_train_predicted = model.predict(x_train) #RMSE on train
y_valid_predicted = model.predict(x_validation) #RMSE on validation


#Metric (RMSE)
rmse_train = rmse(y_train,y_train_predicted)
rmse_val = rmse(y_validation,y_valid_predicted)
print("The RMSE on train is : ",rmse(y_train,y_train_predicted))
print("The RMSE on Validation is : ",rmse(y_validation,y_valid_predicted))

The RMSE on train is :  32981.38358511505
The RMSE on Validation is :  28974.74048634289


In [12]:
filename = 'final_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [14]:
final_test_2 = df_test[cols_to_select]
# filling NA

final_test_2 = fillingNA(final_test_2)
# final_test=SelectHighlyCorr_withTarget(final_test,0.5)


y_pred_test_file = model.predict(final_test_2)
submission  = pd.DataFrame()
submission['Id']=df_test['Id']
submission["SalePrice"]=y_pred_test_file
# submission.info()
submission.to_csv('test_scored.csv', index=False) 